# Base Functions

In [10]:
pip install mpmath

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 133.0 kB/s eta 0:00:0000:01m

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import math, numpy as np, mpmath as mp, pandas as pd, matplotlib.pyplot as plt, plotly.express as px, plotly.graph_objects as go

In [12]:
def taylor_series_expansion(x, max_terms=1000, dps=20):
    """
    Calculates the Taylor series expansion of e^x adaptively 
    with given decimal precision (dps) and up to max_terms.
    
    :param x: The value of x in e^x.
    :param max_terms: Maximum number of terms allowed in the series
    :param dps: Desired decimal precision (sets both mp.dps and epsilon = 1e-dps)
    :return: The approximation of e^x
    """
    mp.mp.dps = dps
    epsilon = mp.mpf(f"1e-{dps}")  # epsilon = 1e-dps
    
    result = mp.mpf(1)  # T0 = 1
    term = mp.mpf(1)
    k = 1
    
    while abs(term) > epsilon * abs(result) and k < max_terms:
        term = term * mp.mpf(x) / k
        result += term
        k += 1
    
    return result

In [13]:
def calculate_ex(x, n : int, precision : int = 20):
    """
    Calculates e^x using the Taylor series expansion and computes the error.
    
    :param x: The value of x in e^x.
    :param n: The number of terms in the Taylor series.
    :param precision: The precision for mpmath calculations (default is 20).
    :return: A tuple containing:
             straight - The direct approximation of e^x.
             inverted - The inverse approximation (1/e^-x) if x is negative, otherwise None.
             real_value - The real value of e^x.
             directError - The error in the direct approximation.
             invertedError - The error in the inverse approximation if applicable, otherwise None.
    """
    mp.mp.dps = precision
    
    real_value = mp.exp(x)
    straight = taylor_series_expansion(x, n, precision)
    directError = mp.mpf(abs(real_value - straight))

    if x < 0:
        inverted = mp.mpf(1) / taylor_series_expansion(-x, n, precision)
        invertedError = mp.mpf(abs(real_value - inverted))
        return straight, inverted, directError, invertedError, real_value
    else:
        return straight, None, directError, None, real_value

In [14]:
def show_ex(x, n : int, precision : int = 20):
    """
    Displays the value of e^x using the Taylor series expansion.
    
    :param x: The value of x in e^x.
    :param n: The number of terms in the Taylor series.
    :param precision: The precision for mpmath calculations (default is 20).
    """
    mp.mp.dps = precision

    straight, inverted, directError, inverseError, real_value = calculate_ex(x, n, precision)

    print(f"\n--------------------------------------------\n")
    print(f"e^x for x = {x} with {n} terms and precision {precision}:\n")
    print(f"\nReal value:                  {real_value} \n")
    print(f"Direct apx:                  {straight}")
    print(f"Direct error:                {directError}")
    if inverted is not None:
        print(f"Inverse apx: (1/e^-x):       {inverted}")
        print(f"Inverse error:               {inverseError}")
        
    print(f"\n--------------------------------------------\n")

In [15]:
def plot_ex(x, n_values : list, precision : int = 20, ):
    """
    Plots the error and approximation of e^x using the Taylor series expansion for different numbers of terms.

    :param x: The value of x in e^x.
    :param n_values: A list of integers representing the number of terms in the Taylor series to evaluate.
    :param precision: The precision for mpmath calculations (default is 20).
    """

    errors = []
    real_values = []
    values = []

    for n in n_values:
        straight_values, inverted_values, directError, inverseError, real_value = calculate_ex(x, n, precision=precision)
        
        if x < 0:
            value = inverted_values
            error = inverseError
        else:
            value = straight_values
            error = directError
        
        errors.append(error)
        real_values.append(real_value)
        values.append(value)

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=list(n_values),
        y=[float(err) for err in errors],
        mode='lines',
        name='Error',
        line=dict(color='red', dash='dashdot')
    ))

    fig.add_trace(go.Scatter(
        x=list(n_values),
        y=[float(rv) for rv in real_values],
        mode='lines',
        name='Real value of e^x',
        line=dict(color='blue', dash='dot')
    ))

    fig.add_trace(go.Scatter(
        x=list(n_values),
        y=[float(val) for val in values],
        mode='lines',
        name='Approximation of e^x',
        line=dict(color='green')
    ))

    fig.update_layout(
        title='Error in Taylor Series Approximation for e^x',
        xaxis_title='Number of Terms (n)',
        yaxis_title='Value',
        #yaxis_range=[- (float(min(values)) * 0.9), float(max(values)) * 1.1],
        #xaxis_range=[min(n_values) * 0.5, max(n_values) * 1.01],
        autosize=True,
        hovermode="x unified",
        showlegend=True,
        #yaxis_type="log"
    )

    fig.show()


In [16]:
show_ex(3, 1000, 100)
show_ex(3, 1000, 12)


--------------------------------------------

e^x for x = 3 with 1000 terms and precision 100:


Real value:                  20.0855369231876677409285296545817178969879078385541501443789342296988458780919737312044971602530177 

Direct apx:                  20.0855369231876677409285296545817178969879078385541501443789342296988458780919737312044971602530177
Direct error:                0.0

--------------------------------------------


--------------------------------------------

e^x for x = 3 with 1000 terms and precision 12:


Real value:                  20.0855369232 

Direct apx:                  20.0855369232
Direct error:                7.27595761418e-12

--------------------------------------------



In [17]:
plot_ex(3, range(1,100), precision=20)

In [18]:
plot_ex(-3, range(1,100), precision=20)

# Tests

## Test A

### Very small x

In [19]:
plot_ex(125, range(1,250), precision=20)

### Very big x

In [20]:
plot_ex(99, range(50,150), precision=50000)

In [21]:
plot_ex(3, range(1,15), precision=1)

In [ ]:
plot_ex(3, range(999,1001), precision=50000)

In [ ]:
plot_ex(3, range(999,1005), precision=50000)